In [1]:
from data_generation import generate_batch

In [4]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import mixture
from scipy import stats
